# Introduction to Threads

## What Is A Thread
- A **Thread Of Execution** is the smallest sequence of programmed instructions that can be managed independently by a **scheduler** and executed by a CPU
- Multiple threads can exist within one process, executing concurrently and sharing resources such as memory
- Each thread has a reserved region of memory referred to as its **stack** 
  - **Local function state** data added to the top of the stack 
  - Used to store the location of a **return address**

![Multithreading](https://upload.wikimedia.org/wikipedia/commons/2/25/Concepts-_Program_vs._Process_vs._Thread.jpg "https://en.wikipedia.org/wiki/File:Concepts-_Program_vs._Process_vs._Thread.jpg")

## Uses

- Responsive user interface
  - Free the main UI thread to continue processing keyboard and mouse events
- Parallel programming
  - Perform intensive calculations faster on multicore or multiprocessor by sharing the workload among multiple threads
- Simultaneous requests processing
  - On a server, handle client requests that arrive concurrently in parallel

## Problems

- Multithreading can increase complexity 
  - The interaction between threads (typically via shared data) can cause intermittent and nonreproducible bugs
- Resource and CPU cost 
  - Scheduling and switching threads when there are more active threads than CPU cores
- Creation/tear-down cost
  - Multithreading will not always speed up your application — it can even slow it down if used excessively or inappropriately

## Every Application starts in a Thread


In [ ]:
using System.Threading;

Console.WriteLine(Thread.CurrentThread.ManagedThreadId); 
Example();

void Example() {
    Console.WriteLine(Thread.CurrentThread.ManagedThreadId);
}

## Creating and starting a new Thread

- We can create a **new Thread**
- We can bind it to a **delegate**
- We can **Start** it  

In [ ]:
using System.Threading;
Console.WriteLine(Thread.CurrentThread.ManagedThreadId); 
Thread t1 = new Thread(Example);
t1.Start();

void Example() {
    Console.WriteLine(Thread.CurrentThread.ManagedThreadId); 
}


## Using Threads

Single Threaded Execution is deterministic: no matter how many times we execute the following examples, we will **always** see the same results 

In [ ]:
using System.Threading;
void printX() {
    for (int i = 0; i < 100; i++) {
        Console.Write("x");
    }
}
//deterministic:
//we will ALWAYS see 100 x
//followed by 100 O
printX();
for (int i = 0; i < 100; i++) {
    Console.Write("O");
}

# Multithreading is non deterministic

Multithreading is like a box of chocolate: you never know what you're going to get

In [ ]:
using System.Threading;
void printX() {
    for (int i = 0; i < 100; i++) {
        Console.Write("x");
    }
}
//non deterministic:
//x and o will mix in a random way
new Thread(printX).Start();
for (int i = 0; i < 100; i++) {
    Console.Write("O");
}


# Lab Time

- Create a **Class Library** with the following function


In [ ]:
using System.Diagnostics;
public void SlowFunction() {
    Stopwatch stopwatch = new Stopwatch();
    stopwatch.Start();
    while (stopwatch.ElapsedMilliseconds < 3000) { }
    Console.WriteLine(42);
  }  

- Create a console application and invoke this function 3 times using 3 different Threads
- You should see the result appearing at the same time, after 3 seconds


# Using Threads - Passing Data

We can pass object to a method during the **Start** of the new Thread

In [ ]:
using System.Threading;
void doYourThing(object o) {
    Console.WriteLine(o);
}
new Thread(doYourThing).Start("a");
new Thread(doYourThing).Start("b");
new Thread(doYourThing).Start("c");

# Lab Time

- Add a new function to your class library

In [ ]:
using System.Diagnostics;
public void SlowSquare01(object o) {             
    int n = (int)o;             
    Stopwatch stopwatch = new Stopwatch();             
    stopwatch.Start();             
    while (stopwatch.ElapsedMilliseconds < (n * 1000)) { }
    Console.WriteLine(n * n);
}  

- Invoke this function 3 times using 3 different Threads and passing 3 different numbers
  - You should see the results appearing at soon as possible

# Using Threads - Passing Data to functions with a different the signature 
We cannot pass data to a Thread if the function has a different signature, for example if it requires more than one parameter or if it has one parameter of a type different than `object`.  
We can, though, use a *trick* with an extra step: 
1. The Thread starts a function without parameters
2. That function invokes the actual function we wanted to invoke in the first place, passing the necessary arguments  
Since we can use *lambda expressions*, it's actually pretty easy

In [ ]:
using System.Threading;
void method01(int a, int b) {
    Console.WriteLine(Thread.CurrentThread.ManagedThreadId);
    Console.WriteLine(a + b);
}
Thread t = new Thread(() => {
    Console.WriteLine(Thread.CurrentThread.ManagedThreadId);
    method01(3, 4);
});
t.Start();

# Lab Time
- Add a function to your class library

In [ ]:
using System.Diagnostics;
public void SlowSquare02(int n) {
    Stopwatch stopwatch = new Stopwatch();
    stopwatch.Start();
    while (stopwatch.ElapsedMilliseconds < (n * 1000)) { }
    Console.WriteLine(n * n);
}  

- Invoke this function 3 times using 3 different Threads and passing 3 different numbers
  - You should see the results appearing at soon as possible

# Foreground and Background Threads
By default, threads you create explicitly are foreground threads.  
Foreground threads keep the application alive for as long as any one of them is running

In [ ]:
using System.Threading;
void go() {
    Console.WriteLine($"Thread {Thread.CurrentThread.ManagedThreadId} running  as a foreground thread: the property Thread.CurrentThread.IsBackground is {Thread.CurrentThread.IsBackground}"); 
}
Thread t = new Thread(go);
t.Start();

Background threads do not keep the application alive.  
Once all foreground threads finish, the application ends, and any background threads still running abruptly terminate.

In [ ]:
using System.Threading;
void go() {
    Console.WriteLine($"Thread {Thread.CurrentThread.ManagedThreadId} running  as a background thread: the property Thread.CurrentThread.IsBackground is {Thread.CurrentThread.IsBackground}"); 
}
Thread t = new Thread(go) { IsBackground = true };
t.Start();

# Exceptions
Catching an exception outside the Thread that threw it is not possible

In [ ]:
using System.Threading;
void go() {
    throw new Exception("ERROR!");
}
try {
    new Thread(go).Start();
} catch (Exception e) {
    Console.WriteLine($"caught {e}"); //unhandled exception
}

Exceptions can be caught by the originating Thread.

In [ ]:
using System.Threading;
void go() {
    throw new Exception("ERROR!");
}
new Thread(()=> {
  try {
    go();
  } catch (Exception ex) { //we can handle it here
    Console.WriteLine($"caught {ex}");
  }
}).Start();

# Returning Values
Threads cannot return values.  
We can put the result in a shared variable but we need to synchronize the threads to avoid concurrency problems such as race conditions.

In [ ]:
using System.Threading;
int method01(int a, int b) {
    Console.WriteLine(Thread.CurrentThread.ManagedThreadId);
    return a + b;
}
int result = 0;
Thread t = new Thread(() => {
    Console.WriteLine(Thread.CurrentThread.ManagedThreadId);
    result = method01(3, 4);
});
t.Start();
t.Join();
Console.WriteLine(result);

36


36


7


# Concurrency Problems
Here is an example of a race condition: we may see `Done!` once or twice, depending on which Thread gets first to `done = true`.

In [ ]:
using System.Threading;
bool done = false;
void doYourThing(){
  if (!done) {
    Console.WriteLine("Done!");
    done = true;
  }
}
//calling the functions in the same thread causes no problem
//but if we call them in different threads we MIGHT get unexpected results
new Thread(doYourThing).Start();
new Thread(doYourThing).Start();

done


done


# Thread Synchronization: lock
The **lock** keyword can help us to synchronize access to resources.

In [ ]:
using System.Threading;
bool done = false;
object lockOnDone = new object();
void doYourThing() {
  lock (lockOnDone) {
    if (!done) {
      Console.WriteLine("done");
      done = true;
    }
  }
}
new Thread(doYourThing).Start();
new Thread(doYourThing).Start();


done


# Lab Time
- Add the following function to your class library

In [ ]:
using System.Diagnostics;
public int SlowSquare03(int n) {             
    Stopwatch stopwatch = new Stopwatch();
    stopwatch.Start();
    while (stopwatch.ElapsedMilliseconds < (n * 1000)) { }
    return n * n;
}  

- Invoke the function 3 times passing 3 different values using different Threads
- Print the results as soon as possible
- Print the sum of the results. Make sure to update the sum in a thread safe way.

# Thread Synchronization - deadlocks
We may cause our threads to *starve* if they keep waiting for a resource that is never released.  
In the following example, if the first Thread gets a lock on `b1` while at the same time the second thread gets a lock on `b2`, the first waits forever on `b2` and the second waits forever on `b1`. Hence, a *deadlock* occurs and our threads *starve*. 

In [ ]:
using System.Threading;
public void Transfer(BankAccount from, BankAccount to, decimal amount) {
    lock (from) {
        lock (to) {
            from.Withdraw(amount);
            to.Deposit(amount);
        }
    }
}

BankAccount b1 = new BankAccount();
BankAccount b2 = new BankAccount();

new Thread(()=>Transfer(b1,b2,100)).Start();
new Thread(()=>Transfer(b2,b1,100)).Start();

public class BankAccount{
    public decimal Balance {get; private set;}
    public void Withdraw(decimal amount) => Balance -= amount;
    public void Deposit(decimal amount) => Balance += amount; 
}

# Thread Synchronization - The Monitor class
The *lock* keyword is actually compiled to more or less this code, which uses the *Enter* and *Exit* methods of the *Monitor* class.

In [ ]:
using System.Threading;
bool done = false;
object lockOnDone = new object();
void doYourThing() {
  try {
    Monitor.Enter(lockOnDone);
    if (!done) {
      Console.WriteLine("Done!");
      done = true;
    }
  } finally {
    Monitor.Exit(lockOnDone);
  }
}
new Thread(doYourThing).Start();
new Thread(doYourThing).Start();

# Thread Syncronization - TryEnter and TryExit
*Enter* and *Exit* wait forever and they are not aware of any deadlock.  
We can use *TryEnter* and *TryExit* to eventually stop waiting.  

In [ ]:
using System.Threading;
bool done = false;
object lockOnDone = new object();
void doYourThing() {
    try {
      //The TryEnter continues after a while if it didn't get a lock on time
      bool lockTaken = Monitor.TryEnter(lockOnDone);
      if (lockTaken) { //let’s check if we got the lock
        if (!done) {
          Console.WriteLine("done");
          done = true;
        }
      } else {
         // Return a boolean to indicate that the operation was not
         // completed successfully and let the caller try again later?
         // Or try yourself again? Both? Decide your strategy.
      }
    } finally {
      Monitor.Exit(lockOnDone);
    }
}
new Thread(doYourThing).Start();
new Thread(doYourThing).Start();

# Thread Syncronization - Interlocked
The Interlocked class provides atomic operations for variables that are shared by multiple threads.  
The following code is not Thread safe, because the `counter` variable is shared by multiple threads and updated concurrrently. 

In [ ]:
using System.Threading;
int counter = 0;
for (int i = 0; i < 10; i++) {
    new Thread(() => counter++).Start();
}

We could rewrite it like this

In [ ]:
using System.Threading;
int counter = 0;
object lockOnCounter = new object();
for (int i = 0; i < 10; i++) {
    new Thread(() => {
        lock(lockOnCounter){
            counter++;
        }
    }).Start();
}

But with the *Interlocked* class we can simplify the code because we can avoid to create our own lock.

In [ ]:
using System.Threading;
int counter = 0;
for (int i = 0; i < 10; i++) {
    new Thread(() => Interlocked.Increment(ref counter)).Start();
}

# Thread Syncronization - AutoResetEvent
In some scenarios we need Threads to send *signals* to each other.  
The **AutoResetEvent** class allows a thread to unblock once, when it receives a signal from another. 

In [ ]:
using System.Threading;
AutoResetEvent autoResetEvent = new AutoResetEvent(false);
new Thread(()=> {
    autoResetEvent.Set();
}).Start();
Console.WriteLine("Waiting for a signal");
autoResetEvent.WaitOne();
Console.WriteLine("Signal received, resuming operations");

# Thread Syncronization - WaitHandle
We can also wait on multiple signals

In [ ]:
EventWaitHandle[] eventWaitHandles = new EventWaitHandle[3] {
    new AutoResetEvent(false), 
    new AutoResetEvent(false), 
    new AutoResetEvent(false)
};
AutoResetEvent autoResetEvent = new AutoResetEvent(false);
new Thread(() => eventWaitHandles[0].Set()).Start();
new Thread(() => eventWaitHandles[1].Set()).Start();
new Thread(() => eventWaitHandles[2].Set()).Start(); 
WaitHandle.WaitAll(eventWaitHandles);
Console.WriteLine("All signals received");

# Lab Time
Modify your previous exercise   
- Use the Interlocked class to update the sum
- Use `AutoResetEvent` and `WaitHandle` to synchronize the Threads

# Thread Syncronization - CountdownEvent
System.Threading.CountdownEvent is a synchronization primitive that unblocks its waiting threads after it has been signaled a certain number of times. CountdownEvent is designed for scenarios in which you would otherwise have to use a ManualResetEvent or ManualResetEventSlim and manually decrement a variable before signaling the event. For example, in a fork/join scenario, you can just create a CountdownEvent that has a signal count of 5, and then start five work items on the thread pool and have each work item call Signal when it completes. Each call to Signal decrements the signal count by 1. On the main thread, the call to Wait will block until the signal count is zero.

In [ ]:
using System.Threading;
CountdownEvent countdown = new CountdownEvent(5);
new Thread(() => countdown.Signal()).Start();
new Thread(() => countdown.Signal()).Start();
new Thread(() => countdown.Signal()).Start();
new Thread(() => countdown.Signal()).Start();
new Thread(() => countdown.Signal()).Start();
countdown.Wait();
Console.WriteLine("I'm here!");

# Thread Synchronization - Semaphore
Threads enter the semaphore by calling the `WaitOne` method, in the case of a `System.Threading.Semaphore` object, or the `SemaphoreSlim.Wait` or `SemaphoreSlim.WaitAsync` method, in the case of a `SemaphoreSlim` object.  
The count on a semaphore is decremented each time a thread enters the semaphore, and incremented when a thread releases the semaphore.  
When the count is zero, subsequent requests block until other threads release the semaphore.  
When all threads have released the semaphore, the count is at the maximum value specified when the semaphore was created.

In [ ]:
using System.Threading;
SemaphoreSlim semaphore = new SemaphoreSlim(3, 3);

for (int i = 0; i < 10; i++) {
    new Thread(() => {
        Console.WriteLine($"{Thread.CurrentThread.ManagedThreadId} waiting. Current count: {semaphore.CurrentCount}");
        semaphore.Wait();
        Console.WriteLine($"{Thread.CurrentThread.ManagedThreadId} working. Current count: {semaphore.CurrentCount}");
        semaphore.Release();
        Console.WriteLine($"{Thread.CurrentThread.ManagedThreadId} released the semaphore. Current count: {semaphore.CurrentCount}");
    }).Start();
}

# Thread Synchronization - Barrier
Another signaling class is the **Barrier**, similar to the CountdownEvent.  
When the CountdownEvent reaches 0, it's done. When the Barrier reaches 0, instead, it restarts its counter.  
The Barrier also allows us to run an *action* everytime the counter reaches 0.

In [ ]:
using System.Threading;
void postPhase(Barrier b) {
    Console.WriteLine("!");
}

//the counter starts from 3
Barrier barrier = new Barrier(3, postPhase);

void go(object letter){
    for (int i = 0; i < 5; i++) {
        Console.Write($"{letter} - {i} ");
        // We decrease the counter and wait until it reaches 0. 
        // We don't move to the next iteration until the other 2 threads have done their job. 
        barrier.SignalAndWait();
    }
}

for (int i = 0; i < 3; i++) {
    new Thread(new ParameterizedThreadStart(go)).Start((char)(i + 65)); //we pass A, B, C
}

# ThreadPool
Everytime we create a `new Thread`, memory is allocated. Whenever a `Thread` has completed its job, we cannot reuse it for something else.  
The **ThreadPool** enables you to use threads more efficiently by providing your application with a pool of worker threads that are managed by the system.     
In the following example, take a look at the `ManagedThreadId`: it may be reused multiple times.
            

In [ ]:
using System.Threading;
void go(object o){
    Console.WriteLine($"{Thread.CurrentThread.ManagedThreadId} got {o}");
}

for (int i = 0; i < 26; i++) {
    ThreadPool.QueueUserWorkItem(go, (char)(i+65));
    //instead of
    //new Thread(go).Start((char)(i+65)); 
}

# Lab Time
Modify your previous exercise by using the `ThreadPool` instead of instancing new `Thread`s.

# Timers
The Time class provides a mechanism for executing a method on a thread pool thread at specified intervals.  
When you create a timer, you can specify an amount of time to wait before the first execution of the method (due time), and an amount of time to wait between subsequent executions (period).   
When a timer is no longer needed, use the Dispose method to free the resources held by the timer.  

In [ ]:
using System.Threading;
using System.Diagnostics;
void go(object _){
    Console.WriteLine($"{Thread.CurrentThread.ManagedThreadId} {Thread.CurrentThread.IsThreadPoolThread}");
}

//wait 3 seconds, than call "go" every second 
using (Timer timer = new Timer(go, null, 3_000, 1_000)) {
    Console.WriteLine(".... Wait for it...");
    Stopwatch stopwatch = new Stopwatch();
    stopwatch.Start();
    // dispose called after 10 seconds
    while (stopwatch.ElapsedMilliseconds < (10_000)) { }
}

# Synchronization Context
